# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data Description](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1. Introduction <a name="introduction"></a>
### 1.1. Background
With population of more than 300,000 Howard County, Maryland, USA is considered as one of the best places to live in USA. Its close proximity to Baltimore and Washington, DC as well as having some of the best schools in country makes it very attractive place of residence for mid to upper income families. Because of this, Howard County is very attractive place for small business such as coffee shops, bakeries, and restaurants. This also makes entry to the market very competitive. For someone, for example opening a bakery, choosing the right location for the shop can make or break the business.

### 1.2. Problem
How someone who wants to open for example a bakery should choose the right location? One would argue that close proximity to the exits from the interstate roads might be a good choice since majority of inhabitants of Howard County is using them for daily commute to their work, so it will make business position in such a location very convenient to stop by. However, Covid-19 and work from home phenomenon dramatically changed people’s daily routines, so location near interstate exit may not be a good choice. It will be useful to conduct geo-location analysis to possibly find pattern and understand which location might be more popular than other. 

### 1.3. Interest
For purpose of this exercise, I’ll will do the analysis and try to predict the best spot to open a bakery. Therefore, this research might be of interest to somebody who wants to open a bakery and needs to know which location might be the best to open such a business.

## 2. Data Description <a name="data"></a>

For the analysis I’ll use **Foursquare API** to get most popular venues in Howard County. Also, I’ll check the location of current bakeries and bakeries within grocery stores. 

While some bakeries within grocery stores are listed on Foursquare, Giant Food’s bakery is not. There is no public data available with this information, so I’ll use **Google Maps** to locate such groceries. I’ll also add grocery stores with no bakeries to our dataset, since opening a new bakery near such a business may not be a bad choice. 

To locate county divisions and get their longitude and latitude I’ll use the **Howard County Zip Codes** from [ciclt.net](http://ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?ClientCode=capitolimpact&State=md&StName=Maryland&StFIPS=24&FIPS=24027). 